In [54]:
import pandas as pd
import sklearn
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
stop = stopwords.words('english')
g_patent = pd.read_csv("output.csv")
g_patent.head()

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
0,Methods for targeting solid tumor stem cells i...,US7850961B2,Abstract\nA small percentage of cells within a...,Classifications\n\nC12N5/0695 Stem cells; Prog...,"[' Michael F', ' ClarkeSean J', ' MorrisonMax ...",\nUniversity of Michigan
1,Monoclonal antibodies for tumor treatment,US9309308B2,Abstract\nA method of treating a tumor or enha...,Classifications\n\nC07K16/18 Immunoglobulins [...,[' Rinat Rotem-YehudarGalina Rodionov '],\nCureTech Ltd
2,Tumor markers in ovarian cancer,US7279294B2,Abstract\nThe present invention features metho...,Classifications\n\nC12Q1/6809 Methods for dete...,"[' Patrice J', ' MorinCheryl A', ' Sherman-Bau...",\nUS Department of Health and Human Services (...
3,Method for optically imaging solid tumor tissue,US5699798A,Abstract\nThe present invention provides a met...,Classifications\n\nA61B5/14553 Measuring chara...,"[' Daryl HochmanMichael M', ' Haglund ']",\nUniversity of Washington
4,Compositions and methods for the diagnosis and...,US7939268B2,Abstract\nThe present invention is directed to...,Classifications\n\nC07K14/4748 Tumour specific...,"[' Gretchen FrantzKenneth J', ' HillanHeidi Ph...",\nGenentech Inc


In [2]:
pattern = ['Abstract\n\n','Abstract\n','\n']
for i in pattern:
    g_patent.Abstract = g_patent.Abstract.str.replace(i,'')

In [3]:
g_patent.head()
g_patent.shape

(4639, 6)

In [4]:
g_patent = g_patent[g_patent.Title != 'Title'] 

In [5]:
g_patent.shape

(4632, 6)

In [6]:
pattern = ['Classifications\n\n','Classifications\n','\n']
for i in pattern:
    g_patent.Classification = g_patent.Classification.str.replace(i,'')
g_patent.head()
g_patent['Title'].head()

0    Methods for targeting solid tumor stem cells i...
1            Monoclonal antibodies for tumor treatment
2                      Tumor markers in ovarian cancer
3      Method for optically imaging solid tumor tissue
4    Compositions and methods for the diagnosis and...
Name: Title, dtype: object

In [7]:
g_patent['Current Assignee'] = g_patent['Current Assignee'].str.replace('\n','')
g_patent.head()

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
0,Methods for targeting solid tumor stem cells i...,US7850961B2,A small percentage of cells within an establis...,C12N5/0695 Stem cells; Progenitor cells; Precu...,"[' Michael F', ' ClarkeSean J', ' MorrisonMax ...",University of Michigan
1,Monoclonal antibodies for tumor treatment,US9309308B2,A method of treating a tumor or enhancing surv...,"C07K16/18 Immunoglobulins [IGs], e.g. monoclon...",[' Rinat Rotem-YehudarGalina Rodionov '],CureTech Ltd
2,Tumor markers in ovarian cancer,US7279294B2,The present invention features methods of diag...,C12Q1/6809 Methods for determination or identi...,"[' Patrice J', ' MorinCheryl A', ' Sherman-Bau...",US Department of Health and Human Services (HH...
3,Method for optically imaging solid tumor tissue,US5699798A,The present invention provides a method and ap...,A61B5/14553 Measuring characteristics of blood...,"[' Daryl HochmanMichael M', ' Haglund ']",University of Washington
4,Compositions and methods for the diagnosis and...,US7939268B2,The present invention is directed to compositi...,C07K14/4748 Tumour specific antigens; Tumour r...,"[' Gretchen FrantzKenneth J', ' HillanHeidi Ph...",Genentech Inc


In [8]:
#g_patent['Inventors'] = g_patent.Inventors.str.replace('[','')
#g_patent['Inventors'] = g_patent.Inventors.str.replace(']','')
#g_patent['Inventors'] = g_patent.Inventors.str.replace('\' ','')
#g_patent['Inventors'] = g_patent.Inventors.str.replace('\'','')

patterns = ['[',']','\' ',' \'','\'']
for i in patterns:
    g_patent['Inventors'] = g_patent.Inventors.str.replace(i,'')


In [9]:
g_patent.head()

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
0,Methods for targeting solid tumor stem cells i...,US7850961B2,A small percentage of cells within an establis...,C12N5/0695 Stem cells; Progenitor cells; Precu...,"Michael F, ClarkeSean J, MorrisonMax S, WichaM...",University of Michigan
1,Monoclonal antibodies for tumor treatment,US9309308B2,A method of treating a tumor or enhancing surv...,"C07K16/18 Immunoglobulins [IGs], e.g. monoclon...",Rinat Rotem-YehudarGalina Rodionov,CureTech Ltd
2,Tumor markers in ovarian cancer,US7279294B2,The present invention features methods of diag...,C12Q1/6809 Methods for determination or identi...,"Patrice J, MorinCheryl A, Sherman-BaustEllen S...",US Department of Health and Human Services (HH...
3,Method for optically imaging solid tumor tissue,US5699798A,The present invention provides a method and ap...,A61B5/14553 Measuring characteristics of blood...,"Daryl HochmanMichael M, Haglund",University of Washington
4,Compositions and methods for the diagnosis and...,US7939268B2,The present invention is directed to compositi...,C07K14/4748 Tumour specific antigens; Tumour r...,"Gretchen FrantzKenneth J, HillanHeidi Phillips...",Genentech Inc


In [10]:
g_patent = g_patent.dropna(how='all', axis=1)
g_patent = g_patent.dropna(how='all', axis=0)

In [11]:
g_patent = g_patent.drop_duplicates(subset = 'Patent Number', keep='first')

In [12]:
g_patent.shape

(4583, 6)

In [13]:
vectorizer = TfidfVectorizer(analyzer='word')
tfidf_matrix = vectorizer.fit_transform(g_patent['Title'].values.astype('U'))
tfidf_feature_name = vectorizer.get_feature_names()

In [14]:
len(tfidf_feature_name)

5796

In [15]:
tfidf_matrix.shape

(4583, 5796)

In [16]:
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [17]:
g_patent = g_patent.reset_index(drop=True)

In [18]:
indices = pd.Series(g_patent['Title'].index)

In [19]:
#Function to get the most similar Patents
def recommend(index, method):
    id = indices[index]
    # Get the pairwise similarity scores of all patents compared to that patent,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(method[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    #Get the patents index
    patents_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar patents using integar-location based indexing (iloc)
    return g_patent['Title'].iloc[patents_index]

In [20]:
#input the index of the patent
recommend(500, cosine_similarity)

306                              Religious game
2306    Nuclear weapon flash protection garment
311              Religious meditation apparatus
3102         Solar combined cycle power systems
3201      Combined Cycle Solar Power Generation
Name: Title, dtype: object

In [21]:
#g_patent['All Content'] = g_patent['Title'] + g_patent['Inventors'] + g_patent['Current Assignee'] + g_patent['Abstract']

In [22]:
#tfidf_all_content = vectorizer.fit_transform(g_patent['All Content'].values.astype('U'))
#tfidf_all_content.shape

In [23]:
# comping cosine similarity matrix using linear_kernal of sklearn
#cosine_similarity_all_content = linear_kernel(tfidf_all_content, tfidf_all_content)
#recommend(33, cosine_similarity_all_content)

In [24]:
#Recommendind Patents based on Title Search
g_patent_data = g_patent[g_patent['Title'].notnull()].copy()
g_patent_data = g_patent_data[g_patent_data['Title'].map(len) >5]

tfidf_des = vectorizer.fit_transform(g_patent_data['Title'].values.astype('U'))
# comping cosine similarity matrix using linear_kernal of sklearn
cosine_sim_des = linear_kernel(tfidf_des, tfidf_des)
#cosine_sim_des
indices_t = pd.Series(g_patent_data['Title'])
#indices_t
inddict_title = indices_t.to_dict()
inddict_title = dict((v,k) for k,v in inddict_title.items())


In [25]:
def recommend_cosine(title):
    id = inddict_title[title]
    # Get the pairwise similarity scores of all patents compared to that patent,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(cosine_sim_des[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    #print(similarity_scores)
      #Get the books index
    patents_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar patents using integar-location based indexing (iloc)
    return g_patent_data.iloc[patents_index]

In [26]:
recommend_cosine("Tumor markers in ovarian cancer")

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
298,Markers for detection of gastric cancer,US10179935B2,Early detection of tumors is a major determina...,C12Q1/6886 Nucleic acid products used in the a...,Parry John GuilfordAndrew John Holyoake,PACIFIC EDGE Ltd
150,"Diagnostic tumor markers, drug screening for t...",US7700715B2,The invention provides a series of composition...,G01N33/54313 Immunoassay; Biospecific binding ...,"Cynthia C, BamdadR, Shoshana Bamdad",Minerva Biotechnologies Corp
274,Renal cell carcinoma tumor markers,US20040096916A1,The present invention relates to tumor markers...,G01N33/57484 Immunoassay; Biospecific binding ...,Roland KellnerKerstin MatzkuBarbara SeligerRud...,Merck Patent GmbH
39,Method of treating colorectal cancer tumor,US8535670B2,The present invention relates to mutations in ...,A61K31/517 Pyrimidines; Hydrogenated pyrimidin...,Somasekar Seshagiri,Genentech Inc
245,Identification and Use of Circulating Nucleic ...,US20160032396A1,Methods for creating a selector of mutated gen...,C12Q1/6886 Nucleic acid products used in the a...,"Maximilian DiehnArash Ash AlizadehAaron M, New...",Leland Stanford Junior University


# Euclidean Distance
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import euclidean_distances

    v = TfidfVectorizer()
    X = v.fit_transform(your_documents)
    D = euclidean_distances(X)
    Now D[i, j] is the Euclidean distance between document vectors X[i] and X[j].

In [27]:
from sklearn.metrics.pairwise import euclidean_distances

In [28]:
#tfidf_des is the vectorizer fn on the Title column
D = euclidean_distances(tfidf_des)

In [29]:
def recommend_euclidean_distance(title):
    ind = inddict_title[title]
    distance = list(enumerate(D[ind]))
    distance = sorted(distance, key=lambda x: x[1])
    distance = distance[1:6]
    #Get the patents index
    patents_index = [i[0] for i in distance]

    #Return the top 5 most similar paptents using integar-location based indexing (iloc)
    return g_patent_data.iloc[patents_index]

In [30]:
recommend_euclidean_distance("Tumor markers in ovarian cancer")

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
298,Markers for detection of gastric cancer,US10179935B2,Early detection of tumors is a major determina...,C12Q1/6886 Nucleic acid products used in the a...,Parry John GuilfordAndrew John Holyoake,PACIFIC EDGE Ltd
150,"Diagnostic tumor markers, drug screening for t...",US7700715B2,The invention provides a series of composition...,G01N33/54313 Immunoassay; Biospecific binding ...,"Cynthia C, BamdadR, Shoshana Bamdad",Minerva Biotechnologies Corp
274,Renal cell carcinoma tumor markers,US20040096916A1,The present invention relates to tumor markers...,G01N33/57484 Immunoassay; Biospecific binding ...,Roland KellnerKerstin MatzkuBarbara SeligerRud...,Merck Patent GmbH
39,Method of treating colorectal cancer tumor,US8535670B2,The present invention relates to mutations in ...,A61K31/517 Pyrimidines; Hydrogenated pyrimidin...,Somasekar Seshagiri,Genentech Inc
245,Identification and Use of Circulating Nucleic ...,US20160032396A1,Methods for creating a selector of mutated gen...,C12Q1/6886 Nucleic acid products used in the a...,"Maximilian DiehnArash Ash AlizadehAaron M, New...",Leland Stanford Junior University


# Pearson's Correlation

In [31]:
#finding similarity using Pearson Corelation
from scipy.stats import pearsonr
tfidf_des_array = tfidf_des.toarray()

In [32]:
def recommend_pearson(title):
    ind = inddict_title[title]
    correlation = []
    for i in range(len(tfidf_des_array)):
        correlation.append(pearsonr(tfidf_des_array[ind], tfidf_des_array[i])[0])
    correlation = list(enumerate(correlation))
    sorted_corr = sorted(correlation, reverse=True, key=lambda x: x[1])[1:6]
    patent_index = [i[0] for i in sorted_corr]
    return g_patent_data.iloc[patent_index]

In [33]:
recommend_pearson("Tumor markers in ovarian cancer")

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
298,Markers for detection of gastric cancer,US10179935B2,Early detection of tumors is a major determina...,C12Q1/6886 Nucleic acid products used in the a...,Parry John GuilfordAndrew John Holyoake,PACIFIC EDGE Ltd
150,"Diagnostic tumor markers, drug screening for t...",US7700715B2,The invention provides a series of composition...,G01N33/54313 Immunoassay; Biospecific binding ...,"Cynthia C, BamdadR, Shoshana Bamdad",Minerva Biotechnologies Corp
274,Renal cell carcinoma tumor markers,US20040096916A1,The present invention relates to tumor markers...,G01N33/57484 Immunoassay; Biospecific binding ...,Roland KellnerKerstin MatzkuBarbara SeligerRud...,Merck Patent GmbH
39,Method of treating colorectal cancer tumor,US8535670B2,The present invention relates to mutations in ...,A61K31/517 Pyrimidines; Hydrogenated pyrimidin...,Somasekar Seshagiri,Genentech Inc
245,Identification and Use of Circulating Nucleic ...,US20160032396A1,Methods for creating a selector of mutated gen...,C12Q1/6886 Nucleic acid products used in the a...,"Maximilian DiehnArash Ash AlizadehAaron M, New...",Leland Stanford Junior University


## So, all 3 methods of similarity: (1) Cosine (2)Euclidean distance (3) Pearson corelation gave us the same resuls.
We are proceeding with cosine similarity. The next funtion takes 2 arguments. 
Argument 1. Title, Author or Current Assignee - The actual value on which the search should be based.
Argument 2. 'Title' or 'Inventors' or 'Assignee' - The user has to specify the argument against which he wants to search.

In [34]:
def recommend_patents(value,criteria):
    if criteria == 'Title':
        tfidf_des = vectorizer.fit_transform(g_patent_data['Title'].values.astype('U'))
        indices = pd.Series(g_patent_data['Title'])
    elif criteria == 'Inventors':
        tfidf_des = vectorizer.fit_transform(g_patent_data['Inventors'].values.astype('U'))
        indices = pd.Series(g_patent_data['Inventors'])
    elif criteria == 'Assignee':
        tfidf_des = vectorizer.fit_transform(g_patent_data['Current Assignee'].values.astype('U'))
        indices = pd.Series(g_patent_data['Current Assignee'])
        
    cosine_sim_des = linear_kernel(tfidf_des, tfidf_des)

    inddict = indices.to_dict()
    inddict = dict((v,k) for k,v in inddict.items())

    id = inddict[value]
    # Get the pairwise similarity scores of all patents compared to that patent,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(cosine_sim_des[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    #print(similarity_scores)
      #Get the books index
    patents_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar patents using integar-location based indexing (iloc)
    return g_patent_data.iloc[patents_index]

In [35]:
recommend_patents('Tumor markers in ovarian cancer','Title')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
298,Markers for detection of gastric cancer,US10179935B2,Early detection of tumors is a major determina...,C12Q1/6886 Nucleic acid products used in the a...,Parry John GuilfordAndrew John Holyoake,PACIFIC EDGE Ltd
150,"Diagnostic tumor markers, drug screening for t...",US7700715B2,The invention provides a series of composition...,G01N33/54313 Immunoassay; Biospecific binding ...,"Cynthia C, BamdadR, Shoshana Bamdad",Minerva Biotechnologies Corp
274,Renal cell carcinoma tumor markers,US20040096916A1,The present invention relates to tumor markers...,G01N33/57484 Immunoassay; Biospecific binding ...,Roland KellnerKerstin MatzkuBarbara SeligerRud...,Merck Patent GmbH
39,Method of treating colorectal cancer tumor,US8535670B2,The present invention relates to mutations in ...,A61K31/517 Pyrimidines; Hydrogenated pyrimidin...,Somasekar Seshagiri,Genentech Inc
245,Identification and Use of Circulating Nucleic ...,US20160032396A1,Methods for creating a selector of mutated gen...,C12Q1/6886 Nucleic acid products used in the a...,"Maximilian DiehnArash Ash AlizadehAaron M, New...",Leland Stanford Junior University


In [36]:
recommend_patents('Genentech Inc','Assignee')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
11,Anti-tumor antibody compositions and methods o...,US7435416B2,The invention provides antibodies targeted at ...,C07K14/4748 Tumour specific antigens; Tumour r...,Brigitte DevauxGilbert-Andre KellerHartmut Koe...,Genentech Inc
19,In vivo tumor detection assay,US5720937A,In vivo assay methods for detecting tumors hav...,"C07K16/32 Immunoglobulins [IGs], e.g. monoclon...","Robert M, HudziakAxel UllrichBrian M, Fendly",Genentech Inc
39,Method of treating colorectal cancer tumor,US8535670B2,The present invention relates to mutations in ...,A61K31/517 Pyrimidines; Hydrogenated pyrimidin...,Somasekar Seshagiri,Genentech Inc
124,Compositions and methods for the diagnosis and...,US7989595B2,The present invention is directed to compositi...,A61K47/6803 Drugs conjugated to an antibody or...,"Mark S, DennisWilliam MalletPaul Polakis",Genentech Inc
155,Antibody compositions for the diagnosis and tr...,US7803915B2,The present invention is directed to compositi...,G01N33/57484 Immunoassay; Biospecific binding ...,Belinda CairnsRuihuan ChenGretchen FrantzKenne...,Genentech Inc


In [37]:
type(g_patent_data.Inventors)

pandas.core.series.Series

In [38]:
g_patent_data.Abstract[124]

'The present invention is directed to compositions of matter useful for the diagnosis and treatment of tumor in mammals and to methods of using those compositions of matter for the same.'

In [39]:
recommend_patents('Michael F, ClarkeSean J, MorrisonMax S, WichaMuhammad Al-Hajj','Inventors')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
2444,Electromagnetic launching system for long-rang...,US4791850A,"An Earth-based, rapid-fire, electromagnetic ac...",F42B6/006 Projectiles for electromagnetic or p...,Michael A,Minovitch
3250,Solar vortex electric power generator,US6772593B2,The SOLAR VORTEX ELECTRIC POWER GENERATOR util...,F03D9/007 Adaptations of wind motors for speci...,Michael A,Dunn
3646,Precooler for an evaporative cooler,US4612778A,An auxiliary cooler shell for installation ove...,F24F5/0035 Air-conditioning systems or apparat...,Michael V,Medrano
3201,Combined Cycle Solar Power Generation,US20130147197A1,Combined cycle solar power generation is achie...,F03G6/04 Devices for producing mechanical powe...,Olaf GoebelYousif Al Ali,Abu Dhabi Future Energy Co PJSC
3121,Adaptive control of a concentrated solar power...,US20110137476A1,A solar-fossil-fuel hybrid power plant is cont...,"G05B13/021 Adaptive control systems, i.e. syst...",Wael Faisal Al-Mazeedi,QGEN Ltd C/O M&C CORPORATE SERVICES


# Next improved function handles the scenariors of random search.
If a Title doesnot exists our function will be able to recommend users based on keywords user searched.

In [67]:
def recommend_patents(value,criteria,numRows):
    df=g_patent_data
    if criteria == 'Title':
        if value not in df['Title']: 
            mod_df = g_patent_data.append({'Title':value},ignore_index=True)
            df = mod_df
        tfidf_des = vectorizer.fit_transform(df['Title'].values.astype('U'))
        indices = pd.Series(df['Title'])
    elif criteria == 'Inventors':
        if value not in df['Inventors']:
            mod_df = g_patent_data.append({'Inventors':value},ignore_index=True)
            df = mod_df
        tfidf_des = vectorizer.fit_transform(df['Inventors'].values.astype('U'))
        indices = pd.Series(df['Inventors'])
    elif criteria == 'Assignee':
        if value not in df['Current Assignee']:
            mod_df = g_patent_data.append({'Current Assignee':value},ignore_index=True)
            df = mod_df
        tfidf_des = vectorizer.fit_transform(df['Current Assignee'].values.astype('U'))
        indices = pd.Series(df['Current Assignee'])
        
    cosine_sim_des = linear_kernel(tfidf_des, tfidf_des)

    inddict = indices.to_dict()
    inddict = dict((v,k) for k,v in inddict.items())

    id = inddict[value]
    # Get the pairwise similarity scores of all patents compared to that patent,
    # sorting them and getting top 5
    similarity_scores = list(enumerate(cosine_sim_des[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    if numRows == '10':
        similarity_scores = similarity_scores[1:11]
    elif numRows == '5':
        similarity_scores = similarity_scores[1:6]
    #print(similarity_scores)
      #Get the patents index
    patents_index = [i[0] for i in similarity_scores]
    
    #Return the top 5 most similar patents using integar-location based indexing (iloc)
    return df.iloc[patents_index]

In [68]:
recommend_patents('Machine Learning Model','Title','5')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
804,Machine learning semantic model,US20140046879A1,The subject technology discloses configuration...,G06N99/005View 1 more classifications,"C, James MacLennanIoan Bogdan Crivat",Predixion Software Inc
651,Machine learning,US20050105712A1,"An automated response system (e.g., an automat...","H04M3/493 Interactive information services, e....",David WilliamsJeffrey Hill,Microsoft Technology Licensing LLC
761,Machine learning,US7249116B2,This invention relates to the machine recognit...,G06K9/6255 Determining representative referenc...,Michael Stephen Fiske,FISKE SOFTWARE LLC
768,Machine learning,US7406452B2,A first set of classifiers are trained to pred...,G06K9/6267 Classification techniquesView 1 mor...,George Henry FormanStephane Chiocchetti,EntIT Software LLC
838,Machine learning,US8250004B2,Computer implemented machine learning methods ...,G06N7/023 Learning or tuning the parameters of...,Plamen Angelov,Lancaster University Business Enterprises Ltd


In [69]:
recommend_patents('Cancer Treatment','Title','10')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
45,Tumor treatment,US5429582A,A completely implantable apparatus is provided...,A61M25/1011 Multiple balloon cathetersView 16 ...,"Jeffery A, Williams",Cytyc Corp
219,Combined cancer treatment methods using antibo...,US7422738B2,Disclosed are the surprising discoveries that ...,A61K49/16 Antibodies; Immunoglobulins; Fragmen...,"Philip E, ThorpeSophia Ran",University of Texas System
152,Implanted tumor cells for the prevention and t...,US6156305A,A method to prevent or treat cancer in a patie...,"A61K9/0024 Solid, semi-solid or solidifying im...","James H, BraukerRobin Lee GellerWilliam D, Joh...",Baxalta GmbH Baxalta Inc
31,Tumor treatment method,US5236410A,A method of treatment of a tumor comprising th...,"A61B18/18 Surgical instruments, devices or met...","Anatoli M, GranovVladimir Y, DerkachDmitry A, ...",Ferrotherm International Inc
877,Methods and systems for monitoring patients un...,US9662049B2,Oncology monitoring systems include: (a) a fir...,A61B5/14539 Measuring characteristics of blood...,"Charles W, ScarantinoH, Troy NagleLester C, Ha...",VTQ IP HOLDING Corp North Carolina State Unive...
267,Tumor treatment apparatus,US5855595A,A tumor treatment apparatus is provided includ...,A61N5/0603 Apparatus for use inside the body f...,Akira FujishimaKazuhito HashimotoYoshinobu Kub...,Pentax Corp Fujishima Akira Kubota Yoshinobu H...
91,Tumor infiltrating lymphocytes as a treatment ...,US5126132A,A new immunotherapeutic method of treating can...,A61K38/2013 IL-2View 1 more classifications,"Steven A, Rosenberg",US Department of Health and Human Services (HHS)
227,Treatment of breast cancer with iodonitrobenza...,US20120269861A1,"In one aspect, the present invention provides ...","A61K31/166 Amides, e.g. hydroxamic acids havin...","Barry M, ShermanCharles BradleyValeria Ossovskaya",BiPar Sciences Inc
279,Methods and compositions for immunotherapy of ...,US20030202985A1,Methods and vaccines for inducing an immune re...,C07K14/005 Peptides having more than 20 amino ...,Yvonne Paterson,University of Pennsylvania
69,Dynamic tumor treatment system,US7343030B2,A workstation imports medical images that depi...,G06T7/0012 Biomedical image inspectionView 6 m...,"Timothy E, Sawyer",ImQuant Inc


In [70]:
recommend_patents('Google','Assignee','5')

,Title,Patent Number,Abstract,Classification,Inventors,Current Assignee
334,"Method, system, and graphical user interface f...",US7831917B1,One aspect of the invention involves a compute...,"G06Q10/10 Office automation, e.g. computer aid...","Joseph F, Karam",Google LLC
350,"Method, system, and graphical user interface f...",US7797642B1,"A method, system, and graphical user interface...",G06Q10/107 Computer aided management of electr...,"Joseph F, KaramDivya Shah",Google LLC
387,Advertiser and user association,US7904461B2,The subject matter of this specification can b...,"G06Q30/02 Marketing, e.g. market research and ...",Shumeet BalujaYushi JingDandapani SivakumarJay...,Google LLC
389,Inferring user interests,US8473500B2,The subject matter of this specification can b...,G06N5/02 Knowledge representationView 2 more c...,Shumeet BalujaYushi JingDandapani SivakumarJay...,Google LLC
411,Campaign in a geo-spatial environment,US8064590B2,"A method, apparatus, and system of a campaign ...","G06Q30/02 Marketing, e.g. market research and ...",Raj Vasant Abhyanker,Google LLC
